In [2]:
import cv2, os
import numpy as np
import glob
import matplotlib.pyplot as plt

In [53]:
import shutil, os

for i in range(100):
    i = 0
    source_idx = i+350
    dst_idx = i
    source_file = os.path.join("/home/yuheng/data/shibuya_640_360_fov45_few_people_bags/2020-08-29-03-56-21/image_0", "%06d.png"%source_idx)
    dst_file = os.path.join("/home/yuheng/data/tartan-air-demo/image_0", "%06d.png"%dst_idx)
    shutil.copy(source_file, dst_file)
    break

# Lucas-Kanade method

In [31]:
import numpy as np


def flow_visualize(flow):
        
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    plt.imshow(bgr)

folder_path = "/home/yuheng/data/tartan-air-demo"
# Enqueue the image list
image_paths = glob.glob(os.path.join(folder_path,"image_0/*.png"))
image_paths.sort()

# Take first frame and find corners in it
old_frame =  cv2.imread(image_paths[0])
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

hsv = np.zeros_like(old_frame)
hsv[...,1] = 255
# Create some random colors
color = np.random.randint(0,255,(100,3))

for i, path in enumerate(image_paths[1:]):
    frame = cv2.imread(path)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    flow = cv2.calcOpticalFlowFarneback(old_gray,frame_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    
    cv2.writeOpticalFlow(os.path.join(folder_path, "flow/%06d.flo"%i), flow)


In [73]:
dpt = cv2.imread("/home/yuheng/data/demo-kitti/depth/000000.png", cv2.IMREAD_UNCHANGED)

In [34]:
disparity_factor = 256.0

left_image_paths = glob.glob(os.path.join(folder_path,"image_0/*.png"))
right_image_paths = glob.glob(os.path.join(folder_path,"image_1/*.png"))
left_image_paths.sort();right_image_paths.sort()

for i, (lim, rim) in enumerate(zip(left_image_paths, right_image_paths)):
    imgL = cv2.imread(lim)
    imgL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    imgR = cv2.imread(rim)
    imgR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
    disparity = stereo.compute(imgL,imgR)
    imd = disparity*disparity_factor
    imd = imd.astype('uint16')
    zero_mask = imd < 0
    imd[zero_mask] = 0
    cv2.imwrite(os.path.join(folder_path, "depth/%06d.png"%i), imd)

# depth to disparity

$$disparity = \frac{BF}{z}$$
$$z = gen/depth_factor$$
$$disparity =  \frac{BF * depth_factor}{z}$$

In [23]:
import glob as glob
import cv2
import numpy as np
import os

folder_path = "/data/tartan-air-demo"

depth_dir = glob.glob("/data/tartan-air-demo/depth_gt/*.png")
depth_dir.sort()
BF = 193.1370849898476
depth_factor = 5000
disparity_factor = 256.0

In [30]:

for depth_path in depth_dir:
    depth = cv2.imread(depth_path,cv2.IMREAD_UNCHANGED)
    disparity = (BF * depth_factor)/depth
    
    file_name = os.path.split(depth_path)[-1]

    disparity*=disparity_factor
    zero_mask =disparity<0
    disparity[zero_mask] = 0
    disparity = disparity.astype('uint16')
    cv2.imwrite(os.path.join(folder_path, "depth/"+file_name), disparity)


/home/yuheng/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
